<a href="https://colab.research.google.com/github/finiteautomata/ml-examples/blob/master/notebooks/tensorflow/newton_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Newton Method in TensorFlow

In this notebook, we will show how to implement Newton's Method to find a real root of a polynomial in TensorFlow

Remember that given a function $f(x)$, the iterative equation of the method is defined as

$$
x_{n+1} = x_n - \frac{f(x)}{f'(x)}
$$

To do this, we will use `tf.map_fn`. This function allows us to do a `map` but in a tensor – remember that we can't just use these functions over symbolic tensors.

We will handle polynomials as 1-d tensors. That is, a tensor

In [0]:
%matplotlib inline
import tensorflow as tf


x = tf.placeholder(tf.float32, shape=(None,))

y = tf.map_fn(lambda x: x+1, x)

with tf.Session() as sess:
  res = sess.run(y, feed_dict={x: [0, 1, 2, 3]})
  print(res)

In [0]:
a = tf.constant([1, 1, 1, 1], dtype=tf.float32)
a_rows = tf.range(tf.shape(a)[0])

a, a_rows

In [3]:

indexes = tf.cast(a_rows, tf.float32)

res, _ = tf.map_fn(lambda x: (x[0] * tf.pow(2.0, x[1]), 1), 
                   (a, indexes), dtype=(tf.float32, tf.float32))

poly_eval = tf.reduce_sum(res)

with tf.Session() as sess:
    print(poly_eval.eval())


15.0


In [4]:
a = tf.placeholder(tf.float32, shape=(None, ))
indexes = tf.range(tf.shape(a)[0])

indexes

<tf.Tensor 'range_1:0' shape=(?,) dtype=int32>

In [5]:
def poly_eval(coeff, x):
  indexes = tf.range(tf.shape(coeff)[0])
  indexes = tf.cast(indexes, tf.float32)
  
  res, _ = tf.map_fn(
      lambda a: (a[0] * tf.pow(x, a[1]), 1),
      (coeff, indexes),
      dtype=(tf.float32, tf.float32)
  )
  
  return tf.reduce_sum(res, axis=0)

coeff = tf.placeholder(tf.float32, shape=(None, ))
x = tf.placeholder(tf.float32)


evaluation = poly_eval(coeff, x)
with tf.Session() as sess:
  s1 = sess.run(evaluation, feed_dict={coeff:[1,1,1,1], x: 2})
  s2 = sess.run(evaluation, feed_dict={coeff:[1,1,1,1], x: [1, 2]})
  
  print(s1)
  print(s2)

15.0
[ 4. 15.]


In [0]:


def poly_der(coefficients):
  """Returns the derivative of a polyomial of coefficients [a_0, ... , a_n]"""
  indexes = tf.range(tf.shape(coeff)[0])
  indexes = tf.cast(indexes, tf.float32)
  
  deriv_coeff = coefficients * indexes 
  return deriv_coeff[1:]



def newton_approx(coefficients, x_0, n_iter=10):
    approx = x_0
    deriv_coeff = poly_der(coefficients)
    for i in range(n_iter):
        approx -= poly_eval(coefficients, approx) / poly_eval(deriv_coeff, approx)
    return approx
  
coeff = tf.placeholder(tf.float32, shape=(None,), name="coeffs")
initial_approx = tf.placeholder(tf.float32, name="initial_approx")

approx = newton_approx(coeff, initial_approx)

In [12]:
with tf.Session() as sess:
  res = approx.eval(feed_dict={coeff:[-2, 0, 1], initial_approx:[5, -10]})
  print(res)

[ 1.4142135 -1.4142135]
